In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    ("placeholder", "{messages}"),
])

model = ChatOpenAI()
chain = prompt | model

chain.invoke({
    "messages": [
        ("human","Translate this sentence from English to French: I love programming."),
        ("ai", "J'adore la programmation."),
        ("human", "What did you just say?"),
    ],
})

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory

chat_history = InMemoryChatMessageHistory()
chat_history.add_user_message("Translate this sentence from English to French: I love programming.")
chat_history.add_ai_message("J'adore la programmation.")
chat_history.messages

In [ ]:
response = chain.invoke({
    "messages": chat_history.messages,
})
chat_history.add_ai_message(response)

input2 = "What did I just ask you?"
chat_history.add_user_message(input2)

chain.invoke({
    "messages": chat_history.messages,
})

# AIMessage(content='You just asked me to translate the sentence "I love programming" from English to French.', response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 61, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5cbb21c2-9c30-4031-8ea8-bfc497989535-0', usage_metadata={'input_tokens': 61, 'output_tokens': 18, 'total_tokens': 79})

In [ ]:
# Automatic history management
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    ("placeholder", "{history}"),
    ("human", "{input}"),
])
chain = prompt | model

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history_for_chain = InMemoryChatMessageHistory()
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history_for_chain,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

# the chain we used before
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer all questions to the best of your ability."),
    ("placeholder", "{history}"),
    ("human", "{input}"),
])
model = ChatOpenAI()
chain = prompt | model

# keep track of history for each combo of user_id and conversation_id
histories: dict[str, InMemoryChatMessageHistory] = {}
def get_session_history(session_id: str = ''):
    if session_id not in histories:
        histories[session_id] = InMemoryChatMessageHistory()
    return histories[session_id]    

# chain with history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

# in action
with_message_history.invoke(
    {"input": "hi im bob!"},
    config={"configurable": {"sesion_id": "123"}},
)
# AIMessage(content='Hello Bob! How can I assist you today?')

# remembers
with_message_history.invoke(
    {"input": "whats my name?"},
    config={"configurable": {"session_id": "123"}},
)
# AIMessage(content='Your name is Bob. How can I help you today, Bob?')

# New session_id --> does not remember
with_message_history.invoke(
    {"input": "whats my name?"},
    config={"configurable": {"session_id": "456"}},
)
# AIMessage(content='I'm sorry, but I don't have access to your personal information such as your name. How can I assist you today?')